# Detecting network intrusions & anomalies with higher-order topological kernels via quantum computation notebook

## 0. Imports and Dependencies

0.0. Dependencies

In [1]:
import os
from quask.core_implementation.qiskit_kernel import QiskitKernel
from quask.core import KernelType, Ansatz, KernelFactory
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler   # added
import numpy as np

/Users/dr.ivanramirez/.pyenv/versions/qml-quask/lib/python3.11/site-packages/pennylane/__init__.py:21: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


0.1. Establish creds if running on IBM hardware (assumes IBM Quantum Cloud account exists and credentials saved with save_account())

In [2]:
# service = QiskitRuntimeService(instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b8ff6077c08a4ea9871560ccb827d457:d3452110-b228-4c79-8959-15ea8cfd435d::") # assuming creds saved with save_account()
# backend = service.backend("ibm_rensselaer")
print("Running on Qiskit simulator (noiseless mode).")

Running on Qiskit simulator (noiseless mode).


## 1. Quantum Backend Configuration

### 1.1. Configure for either quantum simulator or hardware backend

Simulator (noiseless) backend

In [3]:
def create_qiskit_noiseless(ansatz, measurement: str, type: KernelType):
    return QiskitKernel(
        ansatz,
        measurement,
        type,
        platform="Aer",
        n_shots=None
    )

try:
    KernelFactory.add_implementation('qiskit_noiseless', create_qiskit_noiseless)
except ValueError:
    pass

KernelFactory.set_current_implementation('qiskit_noiseless')
print("KernelFactory set to Qiskit noiseless simulator.")

KernelFactory set to Qiskit noiseless simulator.


Hardware (NISQ) backend

Note: only one implementation can be selected (e.g., qiskit_noiseless OR qiskit_ibm)

## 2. Dataset Loading and Preprocessing

### 2.1. Load Dataset

In [4]:
benign_path = "500-benign.npy"
attack_path = "500-attack.npy"

qX1 = np.load(benign_path)
qX2 = np.load(attack_path)

attack_name = os.path.basename(attack_path).replace("attack", "").replace(".npy", "").upper()

print("✅ Loaded datasets:", qX1.shape, qX2.shape)

✅ Loaded datasets: (500, 5) (500, 5)


### 2.2. Feature Count

In [5]:
n_features = qX1.shape[1]
print("✅ Number of features:", n_features)

✅ Number of features: 5


### 2.3 Train/Test Split and Class Balancing

In [6]:
N = 100

qX1 = qX1[:N]
qX2 = qX2[:N]

qX = np.vstack([qX1, qX2])
qy = np.array([-1]*len(qX1) + [1]*len(qX2))

qX_train, qX_test, qy_train, qy_test = train_test_split(
    qX, qy, test_size=0.3, random_state=42, stratify=qy   # ✅ ADDED stratify
)

print("✅ Data split.")

✅ Data split.


### 2.4 Standardization

In [7]:
scaler = StandardScaler()   # ✅ CHANGED
samples = np.append(qX_train, qX_test, axis=0)
scaler.fit(samples)

qX_train = scaler.transform(qX_train)
qX_test = scaler.transform(qX_test)

print("✅ Data standardized.")

✅ Data standardized.


## 3. Quantum Kernel Construction

### 3.1 Define Quantum Ansatz

Config modified from QuASK iris dataset anomaly detection example; see [QuASK: How to optimize a quantum kernel](https://quask.readthedocs.io/en/latest/tutorials_quask/quask_2_optimizers.html) for alternate optimization techniques

In [8]:
ansatz = Ansatz(n_features=n_features, n_qubits=4, n_operations=5)
ansatz.initialize_to_identity()

ansatz.change_operation(0, 0, [0,1], "XX", 2.0)
ansatz.change_operation(1, 1, [1,2], "XY", 2.0)
ansatz.change_operation(2, 2, [2,3], "XZ", 2.0)
ansatz.change_operation(3, 3, [3,0], "YY", 2.0)
ansatz.change_operation(4, 4, [0,2], "YZ", 2.0)

### 3.2 Build Quantum Kernel

In [9]:
kernel = KernelFactory.create_kernel(ansatz, "ZZZZ", KernelType.FIDELITY)
print("✅ Kernel built successfully!")

✅ Kernel built successfully!


## 4. Quantum Model – Baseline Training

### 4.1 Instantiate SVM Classifier

In [10]:
model = SVC(kernel='precomputed', C=10, class_weight='balanced')  
print("✅ SVM upgraded.")

✅ SVM upgraded.


### 4.2 Train Baseline Model

In [11]:
K_train = kernel.build_kernel(qX_train, qX_train)
model.fit(K_train, qy_train)
print("✅ Baseline model trained.")

✅ Baseline model trained.


### 4.3 Baseline Evaluation

In [12]:
K_test = kernel.build_kernel(qX_test, qX_train)
y_pred = model.predict(K_test)

baseline_acc = np.mean(qy_test == y_pred)
print("🎯 BASELINE ACCURACY:", baseline_acc)

🎯 BASELINE ACCURACY: 0.5166666666666667


## 5. Per-Attack Quantum Evaluation (All Matching .npy Pairs)

The above demo handles a singular network attack, split for improved readability. Below is our testing across all attacks.

In [13]:
from sklearn.preprocessing import StandardScaler

data_dir = '.'

for fname in os.listdir(data_dir):
    if fname.endswith('.npy') and 'benign' in fname:
        benign_path = os.path.join(data_dir, fname)
        attack_fname = fname.replace('benign', 'attack')
        attack_path = os.path.join(data_dir, attack_fname)

        attack_label = attack_fname.replace("attack", "").replace("500", "").replace(".npy", "").upper()

        if os.path.exists(attack_path):

            print(f"\n🚨 Evaluating attack: {attack_label}")

            # Load data
            qX1 = np.load(benign_path)[:50]
            qX2 = np.load(attack_path)[:50]

            qX = np.vstack([qX1, qX2])
            qy = np.array([-1]*len(qX1) + [1]*len(qX2))

            # ✅ KEEP CLASS BALANCE
            qX_train, qX_test, qy_train, qy_test = train_test_split(
                qX, qy,
                test_size=0.3,
                random_state=42,
                stratify=qy
            )

            # ✅ USE STRONG NORMALIZATION
            scaler = StandardScaler()
            samples = np.append(qX_train, qX_test, axis=0)
            scaler.fit(samples)

            qX_train = scaler.transform(qX_train)
            qX_test  = scaler.transform(qX_test)

            # ✅ TRAIN
            K_train = kernel.build_kernel(qX_train, qX_train)
            model.fit(K_train, qy_train)

            # ✅ TEST
            K_test = kernel.build_kernel(qX_test, qX_train)
            y_pred = model.predict(K_test)

            accuracy = np.mean(qy_test == y_pred)

            print(f"✅ Accuracy for {attack_label}: {accuracy:.3f}")



🚨 Evaluating attack: -
✅ Accuracy for -: 0.633


In [14]:
model = SVC(kernel="precomputed")

K_train = kernel.build_kernel(qX_train, qX_train)
model.fit(K_train, qy_train)

K_test = kernel.build_kernel(qX_test, qX_train)
y_pred = model.predict(K_test)

baseline = np.mean(y_pred == qy_test)
print("🎯 BASELINE ACCURACY:", baseline)


🎯 BASELINE ACCURACY: 0.6666666666666666


## 6. Hyperparameter Optimization – SVM C

### 6.1 Grid Search Over C

In [15]:
C_values = [0.01, 0.1, 1, 10, 100]
best_C = None
best_acc = 0

print("\n🔍 Tuning SVM parameter C...\n")

for C in C_values:
    model = SVC(kernel="precomputed", C=C)

    K_train = kernel.build_kernel(qX_train, qX_train)
    model.fit(K_train, qy_train)

    K_test = kernel.build_kernel(qX_test, qX_train)
    preds = model.predict(K_test)

    acc = np.mean(preds == qy_test)
    print(f"C = {C:>6} → Accuracy = {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_C = C

print("\n✅ BEST C:", best_C)
print("🚀 TUNED ACCURACY:", best_acc)



🔍 Tuning SVM parameter C...

C =   0.01 → Accuracy = 0.6667
C =    0.1 → Accuracy = 0.6667
C =      1 → Accuracy = 0.6667
C =     10 → Accuracy = 0.6333
C =    100 → Accuracy = 0.7333

✅ BEST C: 100
🚀 TUNED ACCURACY: 0.7333333333333333


## 7. Cross-Validation – Model Stability

### 7.1. K-Fold Cross Validation

In [16]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

print("\n📊 5-fold Cross-validation...\n")

for train_idx, test_idx in kf.split(qX):
    X_tr, X_te = qX[train_idx], qX[test_idx]
    y_tr, y_te = qy[train_idx], qy[test_idx]

    X_tr = scaler.transform(X_tr)
    X_te = scaler.transform(X_te)

    K_tr = kernel.build_kernel(X_tr, X_tr)
    K_te = kernel.build_kernel(X_te, X_tr)

    model = SVC(kernel="precomputed", C=best_C)
    model.fit(K_tr, y_tr)
    preds = model.predict(K_te)

    cv_scores.append(np.mean(preds == y_te))

print("✅ CV MEAN:", np.mean(cv_scores))
print("✅ CV STD :", np.std(cv_scores))



📊 5-fold Cross-validation...

✅ CV MEAN: 0.53
✅ CV STD : 0.10295630140987001
